## can population size changes affect abba baba stats?

imports

In [155]:
import msprime as ms
import numpy as np

### Let's start parameterizing our msprime sims:  
Population size, number of populations (tips on phylogeny, in our case), a migration matrix between populations (zero here -- we're going to use point migrations for simplicity).

## **Important** for interpreting results:  
### We're considering a (((p1,p2)p3)O) tree

In [156]:
Ne = 1000
population_configurations = [
    ms.PopulationConfiguration(sample_size=1, initial_size=Ne)
    for ntip in range(4)]

In [157]:
population_configurations

In [158]:
migmat = [[0,0,0,0],
          [0,0,0,0],
          [0,0,0,0],
         [0,0,0,0]]

### Now define demographic events. 
This includes speciation times (mass migrations where proportion = 1) as well as population size chages (`PopulationParametersChange`) and gene flow (`MassMigration` where proportion is less than 1).  
**These are in backward time**

In [171]:
# no migration, no population size changes
demographic_events = [
    ms.MassMigration(
        time=1000, source=0, destination=1, proportion=1.0),
    ms.MassMigration(
        time=3000, source=1, destination=2, proportion=1.0),
    ms.MassMigration(
        time=6000, source=2, destination=3, proportion=1.0)
]

In [172]:
# this demography debugger is useful for checking that our simulation is doing what we want it to do
dd = ms.DemographyDebugger(
    population_configurations=population_configurations,
    migration_matrix=migmat,
    demographic_events=demographic_events)
dd.print_history()

Epoch: 0 -- 1000.0 generations
     start     end      growth_rate |     0        1        2        3    
   -------- --------       -------- | -------- -------- -------- -------- 
0 |  1e+03    1e+03               0 |     0        0        0        0    
1 |  1e+03    1e+03               0 |     0        0        0        0    
2 |  1e+03    1e+03               0 |     0        0        0        0    
3 |  1e+03    1e+03               0 |     0        0        0        0    

Events @ generation 1000.0
   - Mass migration: lineages move from 0 to 1 with probability 1.0
Epoch: 1000.0 -- 3000.0 generations
     start     end      growth_rate |     0        1        2        3    
   -------- --------       -------- | -------- -------- -------- -------- 
0 |  1e+03    1e+03               0 |     0        0        0        0    
1 |  1e+03    1e+03               0 |     0        0        0        0    
2 |  1e+03    1e+03               0 |     0        0        0        0    
3 |  1e+03  

In [173]:
# now we can make a simulation object
mod=ms.simulate(
            length=100000000,
            num_replicates= 1,  
            mutation_rate=1e-7,
            recombination_rate=1e-8,
            migration_matrix=migmat,
            population_configurations=population_configurations,  
            demographic_events=demographic_events
        )

### Now we can access the genotype matrix

In [174]:
genos=mod.next().genotype_matrix()

In [175]:
genos

array([[1, 1, 1, 0],
       [1, 1, 1, 0],
       [1, 1, 0, 0],
       ...,
       [0, 1, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1]], dtype=uint8)

This matrix tells us where mutations land on the tree under the coalescent w/ recombination

Variants seen here are sampled under the infinite sites model.

So abba and baba sites can just be counted by summing the number of 1,0,1,0 variants and 0,1,1,0 variants

In [176]:
nbaba = np.sum(np.alltrue(genos == np.array([1,0,1,0]),axis=1))
nabba = np.sum(np.alltrue(genos == np.array([0,1,1,0]),axis=1))

In [177]:
print("nbaba = " + str(nbaba))
print("nabba = " + str(nabba))

nbaba = 2100
nabba = 2263


## Now add migration

In [228]:
demographic_events = [
    # here's a pretty huge migration event from p3 to p2, one of the sister taxa
    ms.MassMigration(
        time=800, source=2, destination=1, proportion=.8),
    ms.MassMigration(
        time=1000, source=0, destination=1, proportion=1.0),
    ms.MassMigration(
        time=3000, source=1, destination=2, proportion=1.0),
    ms.MassMigration(
        time=6000, source=2, destination=3, proportion=1.0)
]

In [229]:
# debug again just for kicks
dd = ms.DemographyDebugger(
    population_configurations=population_configurations,
    migration_matrix=migmat,
    demographic_events=demographic_events)
dd.print_history()

Epoch: 0 -- 800.0 generations
     start     end      growth_rate |     0        1        2        3    
   -------- --------       -------- | -------- -------- -------- -------- 
0 |  1e+03    1e+03               0 |     0        0        0        0    
1 |  1e+03    1e+03               0 |     0        0        0        0    
2 |  1e+03    1e+03               0 |     0        0        0        0    
3 |  1e+03    1e+03               0 |     0        0        0        0    

Events @ generation 800.0
   - Mass migration: lineages move from 2 to 1 with probability 0.8
Epoch: 800.0 -- 1000.0 generations
     start     end      growth_rate |     0        1        2        3    
   -------- --------       -------- | -------- -------- -------- -------- 
0 |  1e+03    1e+03               0 |     0        0        0        0    
1 |  1e+03    1e+03               0 |     0        0        0        0    
2 |  1e+03    1e+03               0 |     0        0        0        0    
3 |  1e+03    1

In [230]:
# now we can make a simulation object
mod=ms.simulate(
            length=100000000,
            num_replicates= 1,  
            mutation_rate=1e-7,
            recombination_rate=1e-8,
            migration_matrix=migmat,
            population_configurations=population_configurations,  
            demographic_events=demographic_events
        )
genos=mod.next().genotype_matrix()

In [231]:
nbaba = np.sum(np.alltrue(genos == np.array([1,0,1,0]),axis=1))
nabba = np.sum(np.alltrue(genos == np.array([0,1,1,0]),axis=1))

In [232]:
print("nbaba = " + str(nbaba))
print("nabba = " + str(nabba))

nbaba = 4888
nabba = 6423


This difference makes sense because we migrated a bunch of lineages from p3 to p2.

## Now add populations size changes on either the p1 or p2 lineage

#### First, we'll have a huge bottleneck in the p1 lineage while migrating lineages from p3 to p2.

In [197]:
demographic_events = [
    # what is the population size at this tip at time zero (the others are size defined above)
    # and moving backward in time?
    ms.PopulationParametersChange(
        time=0, initial_size=30, population_id=0),
    # moving backward in time, the pop size of branch now matches the others.
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=1),
    # here's the migration
    ms.MassMigration(
        time=800, source=2, destination=1, proportion=.8),
    # here are the speciation nodes
    ms.MassMigration(
        time=1000, source=0, destination=1, proportion=1.0),
    ms.MassMigration(
        time=3000, source=1, destination=2, proportion=1.0),
    ms.MassMigration(
        time=6000, source=2, destination=3, proportion=1.0)
]
# now we can make a simulation object
mod=ms.simulate(
            length=100000000,
            num_replicates= 1,  
            mutation_rate=1e-7,
            recombination_rate=1e-8,
            migration_matrix=migmat,
            population_configurations=population_configurations,  
            demographic_events=demographic_events
        )
genos=mod.next().genotype_matrix()
nbaba = np.sum(np.alltrue(genos == np.array([1,0,1,0]),axis=1))
nabba = np.sum(np.alltrue(genos == np.array([0,1,1,0]),axis=1))

In [198]:
print("nbaba = " + str(nbaba))
print("nabba = " + str(nabba))

nbaba = 4772
nabba = 7196


So the disproportionate frequency of abba sites is maintained here. 

#### Now, we'll have a huge bottleneck in p2, which is also the recipient of introgression from p3

In [201]:
demographic_events = [
    # what is the population size at this tip at time zero (the others are size defined above)
    # and moving backward in time?
    ms.PopulationParametersChange(
        time=0, initial_size=30, population_id=1),
    # moving backward in time, the pop size of branch now matches the others.
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=1),
    # here's the migration
    ms.MassMigration(
        time=800, source=2, destination=1, proportion=.8),
    # here are the speciation nodes
    ms.MassMigration(
        time=1000, source=0, destination=1, proportion=1.0),
    ms.MassMigration(
        time=3000, source=1, destination=2, proportion=1.0),
    ms.MassMigration(
        time=6000, source=2, destination=3, proportion=1.0)
]
# now we can make a simulation object
mod=ms.simulate(
            length=100000000,
            num_replicates= 1,  
            mutation_rate=1e-7,
            recombination_rate=1e-8,
            migration_matrix=migmat,
            population_configurations=population_configurations,  
            demographic_events=demographic_events
        )
genos=mod.next().genotype_matrix()
nbaba = np.sum(np.alltrue(genos == np.array([1,0,1,0]),axis=1))
nabba = np.sum(np.alltrue(genos == np.array([0,1,1,0]),axis=1))

In [202]:
print("nbaba = " + str(nbaba))
print("nabba = " + str(nabba))

nbaba = 5305
nabba = 7135


Pretty much the same -- multiple runs of this shows variance overlaps pretty entirely.

#### What if p2 grows instead?

In [203]:
demographic_events = [
    # what is the population size at this tip at time zero (the others are size defined above)
    # and moving backward in time?
    ms.PopulationParametersChange(
        time=0, initial_size=10000, population_id=1),
    # moving backward in time, the pop size of branch now matches the others.
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=1),
    # here's the migration
    ms.MassMigration(
        time=800, source=2, destination=1, proportion=.8),
    # here are the speciation nodes
    ms.MassMigration(
        time=1000, source=0, destination=1, proportion=1.0),
    ms.MassMigration(
        time=3000, source=1, destination=2, proportion=1.0),
    ms.MassMigration(
        time=6000, source=2, destination=3, proportion=1.0)
]
# now we can make a simulation object
mod=ms.simulate(
            length=100000000,
            num_replicates= 1,  
            mutation_rate=1e-7,
            recombination_rate=1e-8,
            migration_matrix=migmat,
            population_configurations=population_configurations,  
            demographic_events=demographic_events
        )
genos=mod.next().genotype_matrix()
nbaba = np.sum(np.alltrue(genos == np.array([1,0,1,0]),axis=1))
nabba = np.sum(np.alltrue(genos == np.array([0,1,1,0]),axis=1))

In [204]:
print("nbaba = " + str(nbaba))
print("nabba = " + str(nabba))

nbaba = 5385
nabba = 6660


Still the same.

What if we have multiple population size changes? Growth in the introgressing lineages but bottleneck in the other?

In [205]:
demographic_events = [
    # what is the population size at this tip at time zero (the others are size defined above)
    # and moving backward in time?
    ms.PopulationParametersChange(
        time=0, initial_size=10000, population_id=1),
    ms.PopulationParametersChange(
        time=0, initial_size=30, population_id=0),
    ms.PopulationParametersChange(
        time=0, initial_size=10000, population_id=2),
    # moving backward in time, the pop size of branch now matches the others.
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=1),
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=0),
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=2),
    # here's the migration
    ms.MassMigration(
        time=800, source=2, destination=1, proportion=.8),
    # here are the speciation nodes
    ms.MassMigration(
        time=1000, source=0, destination=1, proportion=1.0),
    ms.MassMigration(
        time=3000, source=1, destination=2, proportion=1.0),
    ms.MassMigration(
        time=6000, source=2, destination=3, proportion=1.0)
]
# now we can make a simulation object
mod=ms.simulate(
            length=100000000,
            num_replicates= 1,  
            mutation_rate=1e-7,
            recombination_rate=1e-8,
            migration_matrix=migmat,
            population_configurations=population_configurations,  
            demographic_events=demographic_events
        )
genos=mod.next().genotype_matrix()
nbaba = np.sum(np.alltrue(genos == np.array([1,0,1,0]),axis=1))
nabba = np.sum(np.alltrue(genos == np.array([0,1,1,0]),axis=1))

print("nbaba = " + str(nbaba))
print("nabba = " + str(nabba))

nbaba = 4820
nabba = 6343


#### Let's try lots of other combinations just for fun.

In [222]:
demographic_events = [
    # what is the population size at this tip at time zero (the others are size defined above)
    # and moving backward in time?
    ms.PopulationParametersChange(
        time=0, initial_size=30, population_id=1),
    ms.PopulationParametersChange(
        time=0, initial_size=10000, population_id=0),
    ms.PopulationParametersChange(
        time=0, initial_size=30, population_id=2),
    # moving backward in time, the pop size of branch now matches the others.
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=1),
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=0),
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=2),
    # here's the migration
    ms.MassMigration(
        time=800, source=2, destination=1, proportion=.8),
    # here are the speciation nodes
    ms.MassMigration(
        time=1000, source=0, destination=1, proportion=1.0),
    ms.MassMigration(
        time=3000, source=1, destination=2, proportion=1.0),
    ms.MassMigration(
        time=6000, source=2, destination=3, proportion=1.0)
]
# now we can make a simulation object
mod=ms.simulate(
            length=100000000,
            num_replicates= 1,  
            mutation_rate=1e-7,
            recombination_rate=1e-8,
            migration_matrix=migmat,
            population_configurations=population_configurations,  
            demographic_events=demographic_events
        )
genos=mod.next().genotype_matrix()
nbaba = np.sum(np.alltrue(genos == np.array([1,0,1,0]),axis=1))
nabba = np.sum(np.alltrue(genos == np.array([0,1,1,0]),axis=1))

print("nbaba = " + str(nbaba))
print("nabba = " + str(nabba))

nbaba = 5187
nabba = 7714


In [219]:
demographic_events = [
    # what is the population size at this tip at time zero (the others are size defined above)
    # and moving backward in time?
    ms.PopulationParametersChange(
        time=0, initial_size=30, population_id=1),
    ms.PopulationParametersChange(
        time=0, initial_size=30, population_id=0),
    ms.PopulationParametersChange(
        time=0, initial_size=10000, population_id=2),
    # moving backward in time, the pop size of branch now matches the others.
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=1),
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=0),
    ms.PopulationParametersChange(
        time=750, initial_size=1000, population_id=2),
    # here's the migration
    ms.MassMigration(
        time=800, source=2, destination=1, proportion=.8),
    # here are the speciation nodes
    ms.MassMigration(
        time=1000, source=0, destination=1, proportion=1.0),
    ms.MassMigration(
        time=3000, source=1, destination=2, proportion=1.0),
    ms.MassMigration(
        time=6000, source=2, destination=3, proportion=1.0)
]
# now we can make a simulation object
mod=ms.simulate(
            length=100000000,
            num_replicates= 1,  
            mutation_rate=1e-7,
            recombination_rate=1e-8,
            migration_matrix=migmat,
            population_configurations=population_configurations,  
            demographic_events=demographic_events
        )
genos=mod.next().genotype_matrix()
nbaba = np.sum(np.alltrue(genos == np.array([1,0,1,0]),axis=1))
nabba = np.sum(np.alltrue(genos == np.array([0,1,1,0]),axis=1))

print("nbaba = " + str(nbaba))
print("nabba = " + str(nabba))

nbaba = 4519
nabba = 6302


#### Very consistent.